# LOAD THE MODEL AND START USING IT

In [2]:
from CatGPT_model import GPT, GPTConfig
import torch
from tokenizers import ByteLevelBPETokenizer
import torch.nn.functional as F

In [3]:
def load_model(model, model_path, device='cpu'):
    """
    Load the model state dictionary from the specified path and load it into the model.

    Parameters:
    model (torch.nn.Module): The model into which the state dictionary will be loaded.
    model_path (str): The path from where the model will be loaded.
    device (torch.device): The device on which the model will be loaded.
    """
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    print(f"Model loaded from {model_path}")

model = GPT(GPTConfig(vocab_size=32768))
model = torch.compile(model)
load_model(model, model_path="../models/CatGPT.pth")

Model loaded from ../models/CatGPT.pth


In [4]:
def generate_text(input_text = 'La intel·ligència artificial tindrà la capactiat de', num_return_sequences = 1, max_length = 100, device='cpu'):

    enc = ByteLevelBPETokenizer(
        '../tokenizer/vocab.json',
        '../tokenizer/merges.txt'
    )

    # Encode the input text
    tokens = enc.encode(input_text).ids
    tokens = torch.tensor(tokens, dtype=torch.long)  # (8,)

    if len(tokens) > max_length:
        max_length = len(tokens) + 50
        print(f"Max length set to {max_length} as input text is longer than the previous max length")
    tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)  # (B, 8)
    x = tokens.to(device)

    # Set manual seed for reproducibility
    torch.manual_seed(42)

    # Generate sequences
    with torch.no_grad():
        for _ in range(max_length - x.size(1)):
            logits, _ = model(x)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, 1)
            x = torch.cat((x, next_token), dim=1)

    # Decode and print the generated sequences
    for i in range(num_return_sequences):
        tokens = x[i].tolist()
        decoded = enc.decode(tokens)
        print(f"Sample {i+1}: {decoded}")

### Examples

In [11]:
generate_text(input_text="El arbres del bosc eren tenebrosos, un somriure es va escoltar i va ser quan", num_return_sequences=3, max_length=50)

Sample 1: El arbres del bosc eren tenebrosos, un somriure es va escoltar i va ser quan va arribar el moment de conèixer tots els habitants d’aquestes muntanyes i es van acostar. Com més preocupats estaven els joves egipcis per la seva situació, més greu
Sample 2: El arbres del bosc eren tenebrosos, un somriure es va escoltar i va ser quan al final ho vam aconseguir de nou... hem gaudit d'aquest moment i seguirem disfrutant amb les vivències cap a casa.
Arribo a casa per fer un
Sample 3: El arbres del bosc eren tenebrosos, un somriure es va escoltar i va ser quan va tocar el públic dempeus se'l va menjar. Havia passat més d'una hora perquè arribés l'hora, tenen dotze o setze anys —va explicar Montes


In [28]:
generate_text(input_text="El sol es ponia en l'horitzó de milers de colors però els pirates no podien", num_return_sequences=3, max_length=50)

Sample 1: El sol es ponia en l'horitzó de milers de colors però els pirates no podien navegar perquè no tenien por i jugaven. Així ho explica el llibre The Wall Street Journal of the TwoBooks, del professor William Shea
Sample 2: El sol es ponia en l'horitzó de milers de colors però els pirates no podien anar ben despistats: "Upítuma era viu" va dir un d'ells, el cap, abraçadíssim pels dos cossos per fer
Sample 3: El sol es ponia en l'horitzó de milers de colors però els pirates no podien desplaçar-se al Carib per a vigilar l'oest. Per això van haver de parar perquè no trobaven cap fallo! Mentre, els altres —els verd


In [31]:
generate_text(input_text="La intel·ligència artificial tindrà la capacitat de", num_return_sequences=3, max_length=50)

Sample 1: La intel·ligència artificial tindrà la capacitat de dissenyar receptes creatives utilitzant mètodes i tècniques poc habituals. No obstant, al desconèixer quines són les més prometedores? Ara veurem si aquesta disciplina podia ser-ne una de nova i podem descobrir-ne la
Sample 2: La intel·ligència artificial tindrà la capacitat de saber quan ho dirà la matemàtica, en aquest cas escollida pel nostre cervell. L’acusava de no tenir consciència, però el consum disparat a partir d’elles li produeix reducció de la intel·ligència
Sample 3: La intel·ligència artificial tindrà la capacitat de desplaçar a un perifèric únicament a una velocitat. Els propers 10 anys celebrarem l'Apol·lo Boreal, tenen sentit científic i és —com ho és el que és— a partir de les
